<a href="https://colab.research.google.com/github/kiran-pyt/classification-PyTorch-and-PyTorch-lightning/blob/main/TheModel3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
from torchsummary import summary

In [2]:

train_transforms=transforms.Compose([
                                      transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))
])
test_transforms=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
])

In [3]:
train=datasets.MNIST('./data',train=True,download=True,transform=train_transforms)
test=datasets.MNIST('./data',train=False,download=True,transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 368242750.36it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 120413214.54it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 165900434.11it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 18713682.48it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
SEED=1
cuda=torch.cuda.is_available()
print('cuda',cuda)
torch.manual_seed(SEED)
if cuda:
  torch.cuda.manual_seed(SEED)

dataloader_args=dict(shuffle=True,batch_size=128,num_workers=4,pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader=torch.utils.data.DataLoader(train,**dataloader_args)
test_loader=torch.utils.data.DataLoader(test,**dataloader_args)




cuda True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
#train_data=train.train_data
#train_data=train.transform(train_data.numpy())
#print('[Train]')
#print('shape',train_data.cpu().numpy().shape)
#print('size',train.train_data.size())
#print('min',torch.min(train_data))
#print('max',torch.max(train_data))
#print('mean',torch.mean(train_data))
#print('std',torch.std(train_data))
#print('var',torch.var(train_data))

In [6]:
dropout_value = 0.1
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.convblock1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=4,kernel_size=(3,3),padding=1,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(4),
                             nn.Dropout(dropout_value)

                             ) # output_size = 28
    self.convblock2=nn.Sequential(nn.Conv2d(in_channels=4,out_channels=8,kernel_size=(3,3),padding=1,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(8),
                             nn.Dropout(dropout_value)
                             ) # output_size = 28
    self.convblock3=nn.Sequential(nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(16),
                             nn.Dropout(dropout_value)
                             )  # output_size = 26

    self.pool1=nn.MaxPool2d(2,2) # output_size = 13

    self.convblock4=nn.Sequential(nn.Conv2d(in_channels=16,out_channels=4,kernel_size=(1,1),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(4),
                             nn.Dropout(dropout_value)
                             )   # output_size = 13
    self.convblock5=nn.Sequential(nn.Conv2d(in_channels=4,out_channels=8,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(8),
                             nn.Dropout(dropout_value)
                             )  # output_size = 11


    self.convblock6=nn.Sequential(nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(16),
                             nn.Dropout(dropout_value)
                             ) # output_size = 9
    self.convblock7=nn.Sequential(nn.Conv2d(in_channels=16,out_channels=10,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(10),
                             nn.Dropout(dropout_value)
                             ) # output_size =7
    #self.convblock8=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=10,kernel_size=(7,7),padding=0,bias=0),
                             #nn.ReLU()
                             #) # nn.ReLU() NEVER!
                               # output_size = 1 7x7x10 | 7x7x10x10 | 1x1x10

    self.gap = nn.Sequential(
           nn.AvgPool2d(kernel_size=7) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1


  def forward(self,x):
    x=self.convblock1(x)
    x=self.convblock2(x)
    x=self.convblock3(x)
    x=self.pool1(x)
    x=self.convblock4(x)
    x=self.convblock5(x)
    x=self.convblock6(x)
    x=self.convblock7(x)
    #x=self.convblock8(x)
    x = self.gap(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=-1)


In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              36
              ReLU-2            [-1, 4, 28, 28]               0
       BatchNorm2d-3            [-1, 4, 28, 28]               8
           Dropout-4            [-1, 4, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             288
              ReLU-6            [-1, 8, 28, 28]               0
       BatchNorm2d-7            [-1, 8, 28, 28]              16
           Dropout-8            [-1, 8, 28, 28]               0
            Conv2d-9           [-1, 16, 26, 26]           1,152
             ReLU-10           [-1, 16, 26, 26]               0
      BatchNorm2d-11           [-1, 16, 26, 26]              32
          Dropout-12           [-1, 16, 26, 26]               0
        MaxPool2d-13           [-1, 16, 13, 13]               0
           Conv2d-14            [-

In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.3007381558418274 Batch_id=468 Accuracy=83.43: 100%|██████████| 469/469 [00:21<00:00, 21.84it/s]



Test set: Average loss: 0.2260, Accuracy: 9492/10000 (94.92%)

EPOCH: 1


Loss=0.21357858180999756 Batch_id=468 Accuracy=94.87: 100%|██████████| 469/469 [00:20<00:00, 22.64it/s]



Test set: Average loss: 0.1074, Accuracy: 9750/10000 (97.50%)

EPOCH: 2


...


Test set: Average loss: 0.0520, Accuracy: 9851/10000 (98.51%)

EPOCH: 13


Loss=0.048810455948114395 Batch_id=468 Accuracy=97.56: 100%|██████████| 469/469 [00:20<00:00, 22.97it/s]



Test set: Average loss: 0.0495, Accuracy: 9849/10000 (98.49%)

EPOCH: 14


Loss=0.06664414703845978 Batch_id=468 Accuracy=97.52: 100%|██████████| 469/469 [00:21<00:00, 22.16it/s]



Test set: Average loss: 0.0477, Accuracy: 9865/10000 (98.65%)

